In [3]:
# To Check GPU
!nvidia-smi

Fri Apr 30 21:50:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive


Mounted at /content/gdrive


In [6]:
# Clone the Darknet to the Colab
#!git clone https://github.com/pjreddie/darknet
#!git clone https://github.com/ansarisam/darknet.git
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 14997 (delta 18), reused 31 (delta 12), pack-reused 14951
Receiving objects: 100% (14997/14997), 13.41 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (10183/10183), done.


In [7]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make
!chmod +x /content/darknet

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [8]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [9]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 8000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [10]:
!mkdir "/mydrive/yolov3/"

mkdir: cannot create directory ‘/mydrive/yolov3/’: File exists


In [ ]:
!unzip /content/gdrive/MyDrive/yolov3/images.zip -d data/obj

In [12]:
# Jet is a class name
!echo "contrail" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3/' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [13]:
!echo "contrail" > data/coco.names

In [14]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-30 21:52:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  76.0MB/s    in 2.0s    

2021-04-30 21:52:27 (76.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [15]:
import glob
images_list = glob.glob("data/obj/images/*.png") + glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210307145111(1)_blur.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210312165614_rotate.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20170902122715_20(1)_contrast.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210312155614(2)_rotate.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20170902135715_blur.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20170902145715_blur.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210312145614_blur.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210315234618_contrast.png', 'data/obj/images/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210314140617_rotate.png', 'data/obj/images/cira-rammb-slider---goes-16--

In [18]:
file_train = open("data/train.txt", "w")   
file_test = open("data/test.txt", "w") 

In [19]:
dataset_path = '/content/darknet/data/obj/images'

In [23]:
counter = 1  
index_test = round(100 / 10)  
for pathAndFilename in images_list:  
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test+1:
        counter = 1
        file_test.write(dataset_path + "/" + title + '.png' + "\n")
    else:
        file_train.write(dataset_path + "/" + title + '.png' + "\n")
        counter = counter + 1

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"/content/darknet/data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [26]:
!./darknet detector calc_anchors /content/darknet/data/obj.data -num_of_clusters 9 -width 416 -height 416

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 416, height = 416 
 read labels from 341 images 
 loaded 	 image: 341 	 box: 625
 all loaded. 

 calculating k-means++ ...

 iterations = 28 


counters_per_class = 625

 avg IoU = 74.74 % 

Saving anchors to the file: anchors.txt 
anchors =  39, 41, 104, 46, 254, 27,  77,103, 202, 50, 167, 84, 340, 46, 296, 75, 267,117
^C


In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show | tee log.txt

In [ ]:
!./darknet detect cfg/yolov3_training.cfg '/content/gdrive/MyDrive/yolov3/yolov3_training_final.weights' '/content/gdrive/MyDrive/yolov3/cira-rammb-slider---goes-16---conus---band_04-opacity-100---20210404175616_20(4)_rotate.png'

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3_training.cfg '/content/drive/MyDrive/yolov3/yolov3_training_final.weights'